In [1]:
#Import torch module and other modules required for the

import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms

from torch.utils.data import Dataset

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [36]:

## Generating Binary radnom digits to pass to the module, using keras 


from tensorflow.keras.utils import to_categorical
### Categorical data to be converted to numeric data
numbers = ["seven", "two", "three", "four", "five", "six", 'one', 'eight', 'nine']

### All Numbers
total_numbers =["one", "two", "three", "four", "five", "six", 'seven', 'eight', 'nine']


## map each number to an integer
mapping = {}
for x in range(len(total_numbers)):
  mapping[total_numbers[x]] = x

# integer representation
for x in range(len(numbers)):
  numbers[x] = mapping[numbers[x]]

one_hot_encode = to_categorical(numbers, dtype='int32')

random_number = next(iter(one_hot_encode))
print(random_number)

[0 0 0 0 0 0 1 0 0]


In [5]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fcr = nn.Linear(in_features=9, out_features=20)
    self.fc1 = nn.Linear(in_features=212, out_features=100)
    self.fc2 = nn.Linear(in_features=100, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=15)
  
  def forward(self, t, random_number):
    # input layer
    x = t

    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4
    
    random_number = torch.tensor(random_number, dtype=torch.float32)
    random_number = self.fcr(random_number)
    random_number = random_number.reshape(1, 20)

    # reshapre
    x = x.reshape(-1, 12 * 4 * 4)

    x = torch.cat((x, random_number), 1) #Concatination of random digit and mnsit digit
    # fc1 layer
    x = self.fc1(x)
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)
    x = F.relu(x)

    # output layer
    x = self.out(x)
    # x = F.softmax(x, dim=1)
    return x

In [6]:
train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform = transforms.Compose([
          transforms.ToTensor()
    ])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [54]:
torch.set_grad_enabled(False)

In [25]:
images = next(iter(train_set))

In [27]:
image, label = images
image.size()

torch.Size([1, 28, 28])

In [37]:
label

5

In [55]:
network = Network()

In [65]:
#Moving our network computation for CPU to GPU, if GPU is available :) 

torch.cuda.is_available()
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

network = Network()
network.to(device)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fcr): Linear(in_features=9, out_features=20, bias=True)
  (fc1): Linear(in_features=212, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=15, bias=True)
)

In [66]:
pred = network(image.unsqueeze(0), random_number)

In [67]:
preds

tensor([[ 0.0906,  0.0609, -0.0205,  0.0510, -0.1084,  0.1029, -0.0429,  0.1057,
         -0.0971,  0.0016, -0.0920,  0.0036, -0.0757,  0.0815, -0.0119]],
       grad_fn=<AddmmBackward>)

In [68]:
F.softmax(preds, dim=1)

tensor([[0.0726, 0.0704, 0.0649, 0.0697, 0.0595, 0.0735, 0.0635, 0.0737, 0.0601,
         0.0664, 0.0604, 0.0665, 0.0614, 0.0719, 0.0655]])

In [69]:
pred.argmax(dim=1)

tensor([2])

In [33]:
# I think we cannot use cross enthropy loss function, i think this funcction can be used when we are doing classification for the batch of images
# However i tried to use NLLoss(nll_loss), as this loss function can be used when we are doing softmax for the output, but couldn't able to get the loss function.

loss = F.cross_entropy(preds, label) # Calculate Loss

loss.backward() # Calculate Gradients
optimizer.step() # Update Weights

print('loss1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, label)
print('loss2:', loss.item())

TypeError: ignored